In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np
import numpy.random as nprand
import pandas as pd
import random
import scipy.stats as spstats
%matplotlib inline

from loomio import *
from socialchoice import *
from timeseries import *

## Combined Evolution Final (Fill Attrition)

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = fill_attrition(df_score)

collection = make_preference_sequence_collection(df_score)

participant_ids = collection.participant_ids()
kendall = TimeSeriesResult()
spearman = TimeSeriesResult()
for stage in range(0, 4):
    for participant_id in participant_ids:
        final_preference = collection[participant_id][3]
        try:
            stage_preference = collection[participant_id][stage]
            r, p = spstats.kendalltau(final_preference.ranks(), stage_preference.ranks())
            kendall.add_sample(stage, r)
            r, p = spstats.spearmanr(final_preference.ranks(), stage_preference.ranks())
            spearman.add_sample(stage, r)
        except:
            continue


In [ ]:
plt.errorbar(range(0,4), kendall.mean(), yerr=kendall.yerr95(), label="Kendall")
plt.errorbar(range(0,4), spearman.mean(), yerr=spearman.yerr95(), label="Spearman")
plt.xticks(range(4))
plt.xlabel('Stage')
plt.ylabel('Agreement')
plt.grid()
plt.legend()
plt.title('Self-Agreement (Combined)')

## Group-Level Evolution (Fill Attrition)

In [ ]:
# Limit to participants who completed all polls
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = fill_attrition(df_score)

df_control = df_score[df_score.treatment == 1]
df_random = df_score[df_score.treatment == 2]

control_collection = make_preference_sequence_collection(df_control)
random_collection = make_preference_sequence_collection(df_random)

participant_ids = control_collection.participant_ids()
kendall_control = TimeSeriesResult()
spearman_control = TimeSeriesResult()
for stage in range(0, 4):
    for participant_id in participant_ids:
        final_preference = control_collection[participant_id][3]
        try:
            stage_preference = control_collection[participant_id][stage]
            r, p = spstats.kendalltau(final_preference.ranks(), stage_preference.ranks())
            kendall_control.add_sample(stage, r)
            r, p = spstats.spearmanr(final_preference.ranks(), stage_preference.ranks())
            spearman_control.add_sample(stage, r)
        except:
            # Missing stage for this participant
            # TODO: Add options for filling in missing data
            continue

participant_ids = random_collection.participant_ids()
kendall_random = TimeSeriesResult()
spearman_random = TimeSeriesResult()
for stage in range(0, 4):
    for participant_id in participant_ids:
        final_preference = random_collection[participant_id][3]
        try:
            stage_preference = random_collection[participant_id][stage]
            r, p = spstats.kendalltau(final_preference.ranks(), stage_preference.ranks())
            kendall_random.add_sample(stage, r)
            r, p = spstats.spearmanr(final_preference.ranks(), stage_preference.ranks())
            spearman_random.add_sample(stage, r)
        except:
            continue

In [ ]:
plt.errorbar(range(0,4), kendall_control.mean(), yerr=kendall_control.yerr95(), label="Control")
plt.errorbar(range(0,4), kendall_random.mean(), yerr=kendall_random.yerr95(), label="Random")
plt.xticks(range(4))
plt.xlabel('Stage')
plt.ylabel('Agreement')
plt.grid()
plt.legend()
plt.title('Self-Agreement (Kendall)')

In [ ]:
plt.errorbar(range(0,4), spearman_control.mean(), yerr=spearman_control.yerr95(), label="Control")
plt.errorbar(range(0,4), spearman_random.mean(), yerr=spearman_random.yerr95(), label="Random")
plt.xticks(range(4))
plt.xlabel('Stage')
plt.ylabel('Agreement')
plt.grid()
plt.legend()
plt.title('Self-Agreement (Spearman)')